# Pytorch vs Tensorflow 

В этой тетрадке мы попробуем посмотреть на разницу между двумя фреймворками, а также заглянуть в будущее и посмотреть на tensorflow 2.0.

Решать будем глупую задачку по конвертации градусов по цельсию в градусы по фаренгейту. По данным будем пытаться восстановить формулу: 

$$ f = c \times 1.8 + 32 $$

In [10]:
import numpy as np

celsius    = np.array([-40, -10,  0,  8, 15, 22,  38],  dtype=float)
fahrenheit = np.array([-40,  14, 32, 46, 59, 72, 100],  dtype=float)

for i,c in enumerate(celsius):
    print("{} degrees Celsius = {} degrees Fahrenheit".format(c, fahrenheit[i]))

-40.0 degrees Celsius = -40.0 degrees Fahrenheit
-10.0 degrees Celsius = 14.0 degrees Fahrenheit
0.0 degrees Celsius = 32.0 degrees Fahrenheit
8.0 degrees Celsius = 46.0 degrees Fahrenheit
15.0 degrees Celsius = 59.0 degrees Fahrenheit
22.0 degrees Celsius = 72.0 degrees Fahrenheit
38.0 degrees Celsius = 100.0 degrees Fahrenheit


# __Главное отличие:__ 

Оба фреймворка работают с тензорами и смотрят на любую модель, как на граф вычислений. Существенная разница заключается в том, что в случае Tensorflow граф определён статистически. Взаимодействие с внешним миром происходит через сессии и плейсхолдеры. Вне сессии мы не можем ничего вычислять. Это усложняет дебаг. 

В случае PyTorch граф определён динамически. Его узлы можно выполнять в любые моменты как угодно и где угодно. 

### Делай раз, Tensorflow 

In [4]:
import tensorflow as tf
tf.__version__

'2.0.0-beta1'

In [6]:
import tensorflow.compat.v1 as tf  # подгружаем первую версию библиотеки 
tf.disable_v2_behavior()           # отключаем функционал второй 
                                   # теперь код, написанный на версии tf 1.x должен работать 
tf.__version__

'2.0.0-beta1'

In [8]:
epochs = 10

# плейсхолдеры для данных 
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)


# параметры модели 
a = tf.Variable(tf.zeros([1]), name='bias')
b = tf.Variable(tf.zeros([1]), name='k')

# модель 
y_hat = b*x + a

# функция потерь и метод оптимизации
loss = tf.sqrt(tf.reduce_sum((y - y_hat)**2))
opt = tf.train.AdamOptimizer(learning_rate = 0.1)

step = opt.minimize(loss)

# процедура обучения 
# открываем вычислительную сессию 
with tf.Session() as sess:
    
    # инициализировали все переменные 
    tf.global_variables_initializer().run() 
    
    for i in range(epochs):
        cur_loss = sess.run(loss, feed_dict={x:celsius, y:fahrenheit})
        print('Текущие потери:', cur_loss)
        
        # шаг оптимизации 
        sess.run(step, feed_dict={x:celsius, y:fahrenheit})
        
    print('\nКоэффициенты:', a.eval()[0], b.eval()[0])
    print('Прогнозы:', sess.run(y_hat, feed_dict={x:[-40,0,38]}))

Текущие потери: 153.62617
Текущие потери: 148.2056
Текущие потери: 142.8606
Текущие потери: 137.60123
Текущие потери: 132.43892
Текущие потери: 127.38669
Текущие потери: 122.45928
Текущие потери: 117.67331
Текущие потери: 113.047386
Текущие потери: 108.60223

Коэффициенты: 1.0066519 0.9923882
Прогнозы: [-38.688873    1.0066519  38.717403 ]


### Делай два, pytorch

In [16]:
import torch
epochs = 10

# плейсхолдеры для данных 
x = torch.from_numpy(celsius).float()
y = torch.from_numpy(fahrenheit).float()

In [17]:
x.numpy()

array([-40., -10.,   0.,   8.,  15.,  22.,  38.], dtype=float32)

In [20]:
# параметры модели 
b = torch.tensor([0.,0.], requires_grad=True) 

# вспомогательный аргумент - сигнал, что надо брать производную по тензору
b

tensor([0., 0.], requires_grad=True)

In [21]:
# модель (обычно задают в виде класса, см. ниже)
def forward(x, b):
    return b[0] + b[1]*x

In [26]:
forward(x, b)

tensor([0., 0., 0., 0., 0., 0., 0.], grad_fn=<AddBackward0>)

In [33]:
# функция потерь и метод оптимизации
opt = torch.optim.Adam([b], lr=0.01)

def loss(pred, target):
    squares = (pred - target) ** 2
    return squares.mean().sqrt()

In [34]:
y_pred = forward(x, b)
y_pred

tensor([0., 0., 0., 0., 0., 0., 0.], grad_fn=<AddBackward0>)

In [35]:
loss_val = loss(y_pred, y)
loss_val

tensor(58.0652, grad_fn=<SqrtBackward>)

In [38]:
loss_val.backward( ) 

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [39]:
b.grad

tensor([ -0.6963, -19.9209])

In [40]:
opt.zero_grad()

In [42]:
# обучение (никаких сессий)
for i in range(epochs):

    y_pred = forward(x, b)
    loss_val = loss(y_pred, y)
    
    loss_val.backward( )  # взяли производную от функции 
    # print(b.grad) # градиент будет храниться внутри тензора!!! 
    
    opt.step() # шаг спуска 
    
    opt.zero_grad() # в пай-торче градиенты накапливаются при вычислении, надо занулять
    # при двух шагах результатом в a.grad будет сумма двух градиентов, поэтому надо его занулять 
    # на каждом шагу 
    
    print('Текущие потери:', loss_val.data.numpy())
    

Текущие потери: 56.017284
Текущие потери: 55.81415
Текущие потери: 55.61136
Текущие потери: 55.408897
Текущие потери: 55.206787
Текущие потери: 55.005028
Текущие потери: 54.80364
Текущие потери: 54.60261
Текущие потери: 54.401966
Текущие потери: 54.201702


In [49]:
b.detach().numpy()

array([0.20051186, 0.19965349], dtype=float32)

In [51]:
forward(40, b)

tensor(8.1867, grad_fn=<AddBackward0>)

In [50]:
print('\nКоэффициенты:', b.detach().numpy())
print('Прогнозы:', forward(40, b).detach().numpy())

# detach() отключает необходимость искать градиент и позволяет перетащить тензор в numpy


Коэффициенты: [0.20051186 0.19965349]
Прогнозы: 8.186651


Обычно модели собираеют в виде классов.

In [52]:
class TorchFarengateNet(torch.nn.Module):
    
    def __init__(self, n_hidden_neurons):
        super(TorchFarengateNet, self).__init__()
        self.fc1 = torch.nn.Linear(1, n_hidden_neurons)
        self.act1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        return x

model = TorchFarengateNet(3)
model

TorchFarengateNet(
  (fc1): Linear(in_features=1, out_features=3, bias=True)
  (act1): Sigmoid()
  (fc2): Linear(in_features=3, out_features=1, bias=True)
)

In [62]:
# плейсхолдеры для данных 
x = torch.from_numpy(celsius).float()
y = torch.from_numpy(fahrenheit).float()
x

tensor([-40., -10.,   0.,   8.,  15.,  22.,  38.])

In [63]:
celsius[:,None]

array([[-40.],
       [-10.],
       [  0.],
       [  8.],
       [ 15.],
       [ 22.],
       [ 38.]])

In [64]:
x.unsqueeze(1) # обычно работают со столбиками

tensor([[-40.],
        [-10.],
        [  0.],
        [  8.],
        [ 15.],
        [ 22.],
        [ 38.]])

In [65]:
x.unsqueeze_(1) #inplace преобразование
y.unsqueeze_(1)

tensor([[-40.],
        [ 14.],
        [ 32.],
        [ 46.],
        [ 59.],
        [ 72.],
        [100.]])

In [66]:
model.forward(x)

tensor([[0.9405],
        [0.9248],
        [0.6001],
        [0.3096],
        [0.2957],
        [0.2937],
        [0.2933]], grad_fn=<AddmmBackward>)

Отдельно задают функцию потерь и оптимизатор. 

In [67]:
def loss(pred, target):
    squares = (pred - target) ** 2
    return squares.mean()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

После циклом запускают обучение.

In [68]:
for epoch_index in range(30):
    optimizer.zero_grad()

    y_pred = model.forward(x)
    loss_val = loss(y_pred, y)

    loss_val.backward()
    print('Текущие потери:', loss_val.data.numpy())
    optimizer.step()

Текущие потери: 3350.0095
Текущие потери: 3348.2947
Текущие потери: 3346.5825
Текущие потери: 3344.8706
Текущие потери: 3343.158
Текущие потери: 3341.4453
Текущие потери: 3339.7332
Текущие потери: 3338.024
Текущие потери: 3336.3171
Текущие потери: 3334.6155
Текущие потери: 3332.9192
Текущие потери: 3331.2288
Текущие потери: 3329.5437
Текущие потери: 3327.8618
Текущие потери: 3326.1782
Текущие потери: 3324.4883
Текущие потери: 3322.7866
Текущие потери: 3321.069
Текущие потери: 3319.3325
Текущие потери: 3317.5667
Текущие потери: 3315.758
Текущие потери: 3313.889
Текущие потери: 3311.938
Текущие потери: 3309.8792
Текущие потери: 3307.683
Текущие потери: 3305.3198
Текущие потери: 3302.768
Текущие потери: 3300.0247
Текущие потери: 3297.111
Текущие потери: 3294.0723


# [Tensorflow 2.0](https://www.tensorflow.org/beta)

Это попытка решить проблему. Фреймворк прошёл альфа-тестирование. Сейчас доступна бета-версия. Поддержка первой версии библиотеки будет постепенно прекращаться. Чтобы попробовать библиотеку:

```
pip3 install tensorflow==2.0.0-beta1 
```

__На всякий случай удалите старую версию tensorflow!!! Иначе при установке могут возникнуть какие-нибудь несовместимости и ошибки.__

Основные отличия: 

* Анексия Keras. Он фактически стал частью библиотеки. 
* Появился режим с eager execution как в pytorch
* Упрощение API путем очистки устаревших API и уменьшения дублирования.


In [69]:
# что происходило в первой версии 
a = tf.constant([1, 2])
b = tf.constant([3, 4])

print(a + b)

Tensor("add_2:0", shape=(2,), dtype=int32)


In [71]:
sess = tf.InteractiveSession()

c = a + b
c.eval()

array([4, 6], dtype=int32)

In [2]:
# тут сделать рестарт тетрадки и заново подгрузить библиотеку 
import tensorflow as tf
tf.__version__

'2.0.0-beta1'

In [3]:
a = tf.constant([1, 2])
b = tf.constant([3, 4])

print(a + b)  # вычисление ребра произошло сразу! 

tf.Tensor([4 6], shape=(2,), dtype=int32)


In [4]:
a.shape

TensorShape([2])

In [5]:
a[1]

<tf.Tensor: id=7, shape=(), dtype=int32, numpy=2>

In [6]:
b**2

<tf.Tensor: id=10, shape=(2,), dtype=int32, numpy=array([ 9, 16], dtype=int32)>

In [7]:
a.numpy()

array([1, 2], dtype=int32)

In [ ]:
# Рубрика эксперименты! 